## Product Sentiment

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00


Loading hugging face token

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
product_data = load_dataset("dipawidia/ecommerce-product-reviews-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11606 [00:00<?, ? examples/s]

In [ ]:
product_data

DatasetDict({
    train: Dataset({
        features: ['review', 'translate', 'sentimen'],
        num_rows: 11606
    })
})

In [ ]:
product_df = product_data["train"].to_pandas()
product_df.head()

,review,translate,sentimen
0,Suara dan mic puas. Hanya saya pesan yg nyala ...,sound and mic satisfied. I only ordered the on...,0
1,Good. Mudah-mudahan awet. Makasih,good. hopefully durable. Thanks,1
2,"kondisi dus robek2, tolong di perhatikan lagi","the condition of the box is torn, please pay a...",0
3,Kualitas produk baik. Harga terjangkau dan pen...,product quality is good. affordable prices and...,0
4,"Overall bagus, tombol berfungsi semuanya, tapi...","overall is good, the buttons all work, but the...",0


In [ ]:
features = product_data["train"].features
features

{'review': Value(dtype='string', id=None),
 'translate': Value(dtype='string', id=None),
 'sentimen': Value(dtype='int64', id=None)}

In [ ]:
product_df.shape

(11606, 3)

Cleaning data

In [ ]:
product_df.drop(columns = "review", axis = 1, inplace = True)
product_df.head()

,translate,sentimen
0,sound and mic satisfied. I only ordered the on...,0
1,good. hopefully durable. Thanks,1
2,"the condition of the box is torn, please pay a...",0
3,product quality is good. affordable prices and...,0
4,"overall is good, the buttons all work, but the...",0


Renaming columns

In [ ]:
product_df.rename(columns = {"translate":"review", "sentimen": "sentiment"}, inplace = True)

In [ ]:
product_df.head()

,review,sentiment
0,sound and mic satisfied. I only ordered the on...,0
1,good. hopefully durable. Thanks,1
2,"the condition of the box is torn, please pay a...",0
3,product quality is good. affordable prices and...,0
4,"overall is good, the buttons all work, but the...",0


In [ ]:
product_df.isna().sum()

,0
review,0
sentiment,0


In [ ]:
product_df["sentiment"].value_counts()

,count
sentiment,
0,5803
1,5803


Splitting data into train and test data

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(product_df, test_size = 0.2, stratify = product_df["sentiment"],random_state = 42)

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.head()

,review,sentiment
0,"Thank God, hopefully it will last",1
1,the pot bounces,0
2,very long delivery. you have to set it yoursel...,0
3,"admin doesn't respond, the goods are really go...",0
4,Delivery of goods is delayed due to restocked ...,0


In [ ]:
train["sentiment"].value_counts()

,count
sentiment,
1,4642
0,4642


In [ ]:
test.head()

,review,sentiment
0,"goods okay. great, fast delivery. cm, there is...",0
1,the product has been well received and accordi...,0
2,"original item, long delivery due to anteraja s...",0
3,tablet + accessories well received and accordi...,1
4,"usb can only play mp (mp can't), what's surpri...",0


In [ ]:
test["sentiment"].value_counts()

,count
sentiment,
0,1161
1,1161


In [ ]:
from transformers import AutoTokenizer

model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train['review']

,review
0,"Thank God, hopefully it will last"
1,the pot bounces
2,very long delivery. you have to set it yoursel...
3,"admin doesn't respond, the goods are really go..."
4,Delivery of goods is delayed due to restocked ...
...,...
9279,the sound is really great. the bass is top mar...
9280,"the delivery took really long, more than the e..."
9281,great item. aayang long delivery. please fix it
9282,product quality is very good product price is ...


Converting the data back to hugging face dataset

In [ ]:
from datasets import Dataset

In [ ]:
# importtant step convert vack to hugging face dataset
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [ ]:
train_dataset = Dataset.from_pandas(train.rename(columns={'sentiment': 'labels'}))  # model default name trainer expects is labels
test_dataset = Dataset.from_pandas(test.rename(columns={'sentiment': 'labels'}))

In [ ]:
# Format the dataset to output PyTorch tensors
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataset

Dataset({
    features: ['review', 'labels'],
    num_rows: 9284
})

In [ ]:
test_dataset

Dataset({
    features: ['review', 'labels'],
    num_rows: 2322
})

Pre Processing of the data

In [ ]:
def tokenize_function(data):
    return tokenizer(data['review'], truncation=True)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9284 [00:00<?, ? examples/s]

Map:   0%|          | 0/2322 [00:00<?, ? examples/s]

In [ ]:
train_tokenized

Dataset({
    features: ['review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9284
})

In [ ]:
test_tokenized

Dataset({
    features: ['review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2322
})

Creating Evaluate metric

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def get_metric(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return accuracy.compute(predictions = predictions, references = labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

Creating label for model to understand

In [ ]:
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

Initializing the model

In [ ]:
model = "bert-base-uncased"
MODEL = AutoModelForSequenceClassification.from_pretrained(model, num_labels = 2, id2label = id2label, label2id = label2id)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training the Model

In [ ]:
training_args = TrainingArguments(
    output_dir="product-reviews-sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",  # Updated from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model = MODEL,
    args = training_args,
    train_dataset = train_tokenized,
    eval_dataset = test_tokenized,
    tokenizer = tokenizer,
    compute_metrics = get_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.277600,0.172615,0.937554
2,0.138400,0.204632,0.944014


TrainOutput(global_step=1162, training_loss=0.19470059235617135, metrics={'train_runtime': 383.7083, 'train_samples_per_second': 48.391, 'train_steps_per_second': 3.028, 'total_flos': 799701386105280.0, 'train_loss': 0.19470059235617135, 'epoch': 2.0})

In [ ]:
from transformers import AutoTokenizer

model_1 = "microsoft/MiniLM-L12-H384-uncased"
tokenizer_1 = AutoTokenizer.from_pretrained(model_1)

In [ ]:
def tokenize_function(data):
    return tokenizer_1(data['review'], truncation=True)

train_tokenized_1 = train_dataset.map(tokenize_function, batched=True)
test_tokenized_1 = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9284 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2322 [00:00<?, ? examples/s]

In [ ]:
train_tokenized_1

Dataset({
    features: ['review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9284
})

In [ ]:
test_tokenized_1

Dataset({
    features: ['review', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2322
})

In [ ]:
model_1 = "microsoft/MiniLM-L12-H384-uncased"
MODEL_1 = AutoModelForSequenceClassification.from_pretrained(model_1, num_labels = 2, id2label = id2label, label2id = label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="product-reviews-sentiment_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",  # Updated from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model = MODEL_1,
    args = training_args,
    train_dataset = train_tokenized_1,
    eval_dataset = test_tokenized_1,
    tokenizer = tokenizer_1,
    compute_metrics = get_metric
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.303800,0.197341,0.931525
2,0.185400,0.180536,0.944875


TrainOutput(global_step=1162, training_loss=0.2326911919703787, metrics={'train_runtime': 138.9063, 'train_samples_per_second': 133.673, 'train_steps_per_second': 8.365, 'total_flos': 200213987606976.0, 'train_loss': 0.2326911919703787, 'epoch': 2.0})

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis",model = "jacobasir/product-reviews-sentiment_1")
classifier("Fantastic product! Exceeded my expectations. Quality is top-notch and customer service is excellent. Highly recommended.")

model.safetensors:  50%|#####     | 220M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.9982724189758301}]

In [ ]:
classifier = pipeline("sentiment-analysis",model = "jacobasir/product-reviews-sentiment_1")
classifier("Disappointing purchase. The product didn't meet expectations and customer service was poor. Would not recommend")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9968164563179016}]

In [ ]:
classifier = pipeline("sentiment-analysis",model = "jacobasir/product-reviews-sentiment_1")
classifier("This product is a game-changer! It's user-friendly, durable, and has made my daily tasks so much easier. I genuinely can't recommend it enough.")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'POSITIVE', 'score': 0.990477442741394}]

In [ ]:
classifier = pipeline("sentiment-analysis",model = "jacobasir/product-reviews-sentiment_1")
classifier("Not happy with this product at all. It's not as described, and I had a lot of issues while using it. The company needs to work on their quality control and customer service")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9966543912887573}]